In [1]:
import os
import pandas as pd
import json

In [2]:
website_info = pd.read_csv('./../data/Website - Website.csv')
website_info.tail()

,First Name,Last Name,Professional Headshot,Funny Headshot,GitHub,LinkedIn,PDF Resume,Is Job Searching,Capstone (link),Personal Site,Website Portfolio,Tagline,Bio
21,William,McWilliams,x,NaN,https://github.com/WMcWilliams15,https://www.linkedin.com/in/will-mcwilliams/,x,True,https://github.com/WMcWilliams15/NSS-Capstone,NaN,https://github.com/WMcWilliams15/NSS-Capstone,“It always seems impossible until it is done.”...,"Carpenter turned data analyst, who has an infi..."
22,Yasin,Hasan,x,NaN,https://github.com/yhasan47,https://www.linkedin.com/in/yasinhasan47,NaN,True,https://github.com/yhasan47/nfl-combine-capstone,NaN,https://github.com/yhasan47/nfl-combine-capstone,"""I am ready to face any challenge that might b...",A Science Teacher turned into a Data Analyst.
23,UrLeaka,Test,x,x,https://github.com/unewsome,https://www.linkedin.com/in/urleaka-newsome/,NaN,NaN,NaN,NaN,NaN,"""Data is as data does""",This stuff is fun!' I can't believe I said tha...
24,Matt,Test,x,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25,Taylor,Test,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"""Data does as data is""",NaN


---

Filter out the teachers / nan stuff

In [3]:
website_info = website_info[~((website_info['Last Name'] == 'Test') | website_info['First Name'].isnull())]

In [4]:
website_info.head()

,First Name,Last Name,Professional Headshot,Funny Headshot,GitHub,LinkedIn,PDF Resume,Is Job Searching,Capstone (link),Personal Site,Website Portfolio,Tagline,Bio
0,Adam,Tsuchiyama,X,NaN,https://github.com/adamtsuch21,https://www.linkedin.com/in/adam-tsuchiyama/,x,True,https://github.com/adamtsuch21/nss_capstone,NaN,https://github.com/adamtsuch21/nss_capstone,"""In God we trust. All others, bring data."" - W...",Whether it's determining how to incorporate th...
1,Alan,Brunlinger,x,X,https://github.com/abrunlinger,https://www.linkedin.com/in/alan-brunlinger/,X,True,https://github.com/abrunlinger/Quarantine-Life...,NaN,https://github.com/abrunlinger/Quarantine-Life...,"“Any fool can make things complicated, it requ...",I have always had a passion for deciphering da...
2,Ann,Rumsey,x,NaN,https://github.com/annrinTN,https://www.linkedin.com/in/ann-rumsey,x,False,https://github.com/annrinTN/capstone_annrumsey,https://annrintn.github.io/,https://annrintn.github.io/,Connecting the dots between people and data,Voracious curiosity compels me to study patter...
3,Brandes,Moore,x,X,https://github.com/brandesmoore/,https://www.linkedin.com/in/brandesmoore,x,True,https://github.com/brandesmoore/capstone_track...,NaN,https://github.com/brandesmoore/capstone_track...,"""Every day I wake up is another chance to get ...",My data journey has led me to question everyth...
4,Brant,Ivey,x,X,https://github.com/BrantIvey,https://www.linkedin.com/in/brant-ivey/,x,True,https://github.com/BrantIvey/NSS_Capstone,NaN,https://github.com/BrantIvey/NSS_Capstone,“People...operate with beliefs and biases. To ...,"When someone asks a question, I want to know m..."


---

Rename the columns to something that can be ingested by the js

In [5]:
column_map = {
    'First Name': 'firstName',
    'Last Name': 'lastName',
    'Tagline': 'reelThemIn',
    'Bio': 'bio',
    'GitHub': 'github',
    'LinkedIn': 'linkedin',
    'Is Job Searching': 'job_searching',
    'Website Portfolio': 'portfolio'
}

website_info = website_info.rename(column_map, axis=1)
website_info = website_info[column_map.values()]

website_info.columns

Index(['firstName', 'lastName', 'reelThemIn', 'bio', 'github', 'linkedin',
       'job_searching', 'portfolio'],
      dtype='object')

---

Make the paragraphs

In [6]:
def html_paragraph(v):
    if pd.isna(v):
        return ""
    return f"<p>{str(v)}</p>"


website_info['reelThemIn'] = website_info['reelThemIn'].apply(html_paragraph)
website_info['bio'] = website_info['bio'].apply(html_paragraph)

---

Clean up some excess characters

In [7]:
for col in website_info.columns:
    try:
        website_info[col] = website_info[col] \
            .str.replace('“', '"') \
            .str.replace('”', '"') \
            .str.replace("’", "'") \
            .str.replace("—", "-")
    except:
        pass

---

Gather the image paths

In [8]:
def image_path(name, idx):
    return f"../assets/img/resized/{name.lower()}{idx}.jpeg"


website_info['proImg'] = website_info['firstName'].apply(image_path, idx=1)
website_info['funImg'] = website_info['firstName'].apply(image_path, idx=2)

---

Resume paths

In [9]:
def resume_path(name):
    path = f"../assets/resume/{name.lower()}.pdf"
    if os.path.exists(path):
        return path


website_info['resume'] = website_info['firstName'].apply(resume_path)
website_info[['firstName', 'resume']]

,firstName,resume
0,Adam,../assets/resume/adam.pdf
1,Alan,../assets/resume/alan.pdf
2,Ann,../assets/resume/ann.pdf
3,Brandes,../assets/resume/brandes.pdf
4,Brant,../assets/resume/brant.pdf
5,Christina,../assets/resume/christina.pdf
6,David,../assets/resume/david.pdf
7,Elissa,None
8,Erin,../assets/resume/erin.pdf
9,Fitsum,../assets/resume/fitsum.pdf


---

Assign Ids

In [10]:
website_info['id'] = website_info.index + 1

---

Take a peek

In [11]:
website_info.head(1)

,firstName,lastName,reelThemIn,bio,github,linkedin,job_searching,portfolio,proImg,funImg,resume,id
0,Adam,Tsuchiyama,"<p>""In God we trust. All others, bring data."" ...",<p>Whether it's determining how to incorporate...,https://github.com/adamtsuch21,https://www.linkedin.com/in/adam-tsuchiyama/,True,https://github.com/adamtsuch21/nss_capstone,../assets/img/resized/adam1.jpeg,../assets/img/resized/adam2.jpeg,../assets/resume/adam.pdf,1


In [16]:
import json
json.dumps(website_info.to_dict(orient='records'), )

[{'firstName': 'Adam',
  'lastName': 'Tsuchiyama',
  'reelThemIn': '<p>"In God we trust. All others, bring data." - W. Edwards Deming</p>',
  'bio': "<p>Whether it's determining how to incorporate the current inventory into a menu, or planning a menu based off student feedback, I've always enjoyed using critical thinking and analysis to solve problems. Through my experience at Nashville Software School, I've learned how to apply those same critical thinking skills to data analytics.</p>",
  'github': 'https://github.com/adamtsuch21',
  'linkedin': 'https://www.linkedin.com/in/adam-tsuchiyama/',
  'job_searching': True,
  'portfolio': 'https://github.com/adamtsuch21/nss_capstone',
  'proImg': '../assets/img/resized/adam1.jpeg',
  'funImg': '../assets/img/resized/adam2.jpeg',
  'resume': '../assets/resume/adam.pdf',
  'id': 1},
 {'firstName': 'Alan',
  'lastName': 'Brunlinger',
  'reelThemIn': '<p>"Any fool can make things complicated, it requires a genius to make things simple" ― E.F. S

In [13]:
?website_info.to_json